# Working with rasters

This is the notebook to start with in this repository.  After this notebook move on to any of the notebooks in this folder.

### Table of Contents
* 🧠Anatomy of a raster
* 📖Opening a file
* 🔍Inspecting the metadata
* 🚜Working with the data
* 💾Saving a file

## 🧠 The Anatomy of a raster

Working with raster data involves understanding how rasters are organized.

<img src="./images/raster_anatomy.jpg" width="550">

### 📝 Checking In

You open a raster file and see that the count is 224.  What have you just learned about the raster?

## 📖 Opening a file

When you open data files in Python you usually need to use a library.  When dealing with raster data there are several common libraries, most of which cater to a specific data type:

  
|  File Format |  File Extension | Python Library  |
|---|---|---|
|  netCDF |  `.nc` | netCDF4  |
|  HDF5 | `.hdf5`  | h5py  |
| HDF4 | `.hdf4` | pyhdf |
| geoTIFF | `.tiff` | rasterio |
| zarr | `.zarr` | zarr |


While each of these libraries works best with their associated file type, I really like to use `rasterio` because it does a particularly good job of accomodating [many data types](https://gdal.org/drivers/raster/index.html).  I usually start with `rasterio` and switch to the a different file-specific library only if I need something extra `rasterio` doesn't already provide.

### Opening a file with `rasterio`:

We open our rasters with the `rasterio.open()` method.  The general syntax is 
> ```python
> with rasterio.open(FILEPATH_TO_RASTER, DATAMODE) as src:
    print(src)
> ```

where `FILEPATH_TO_RASTER` is the place on your computer where the data is stored and `DATAMODE` is usually either 'r' for read (opening existing data) or 'w' for write (creating new data).  There are other data modes but these are two very common ones.

An example of opening a file using test AVIRIS data:

In [1]:
# Define the filepath to our raster
# filepath_rad = './data/subset_f180628t01p00r02_corr_v1k1_img'

### Filepaths
Ahe **filepath** and is a string that describes the location of the data that you want to open.  A few pieces of the anatomy of a filepath to notice:
* `/` - forward slashes signal that you have entered a new folder.
* `.` - the period at the beginning tells the computer to start looking for data in the same place tht the code is being run in.  

Choosing to start your filepath with a `.` is called specificying a **relative filepath**, because you are telling the computer to start looking for the file relative to where the file is being run. If you move this file to another place on your computer and don't move the data with it the import statment won't work anymore.  The alternative to a relative filepath is an **aboslute filepath**, in which case you start your file path at the very tippy top of your computer's organizational structure (the root directory).

Other vocab notes:
* **directory** is the same thing as a folder.

To loop back to our example, we put together our filepath by defining the following directions for our computer:
1. start by specifing the current directory as the starting point: `.`
2. go into the data folder: `./data`
3. choose the file named "subset_f180628t01p00r02_corr_v1k1_img": 
`'./data/subset_f180628t01p00r02_corr_v1k1_img'`

🎉 And there we have our file

In [2]:
# import ra/sterio

In [3]:
# Open the file
# with rasterio.open(filepath_rad, 'r') as src:
#     print(src)

The object that gets returned here `<open DatasetReader name='./data/subset_f180628t01p00r02_corr_v1k1_img' mode='r'>` is the "Dataset Reader" object.  This object connects you to the file in Python, but it isn't actually the data.  To get to the data you need to use `src.read()`.

### Reading a raster band with `rasterio`:

General syntax:

> ```python
> with rasterio.open(FILEPATH_TO_RASTER, DATAMODE) as src:
    print(src.read(BAND_INDEX))
> ```

where `FILEPATH_TO_RASTER` is the place on your computer where the data is `DATAMODE` is usually either 'r' for read (opening existing data) or 'w' for write (creating new data).  `BAND_INDEX` indicates which band you want to read.  Leaving `BAND_INDEX` blank reads the entire raster (all bands).

An example:

In [4]:
# Reading the first band from our file
# with rasterio.open(filepath_rad, 'r') as src:
#     print(src.read(5))

What gets printed here are our actual data values! 🎉

### Getting a quick visual of the data

Another way to quickly get eyes on our data is to use the `show` command.

In [5]:
# from matplotlib import pyplot

In [6]:
# with rasterio.open(filepath_rad, 'r') as src:
#     pyplot.imshow(src.read(5))

The plot and the array might not be very useful right now, but at least we have validated that we can open our raster and that it contains data.

If you like viewing quick plots as a way of interacting with your data you can read more about it in the [rasterio docs](https://rasterio.readthedocs.io/en/latest/topics/plotting.html).

### 🌀 Extra background - Context managers

It might be new to you that I used the syntax:
```python
with ... as src:
    src.read()
```

This syntax is called the **Context manager**.  When we read files we are opening them up, like a book, and they stay open until we tell the program to close them.  The benefit of the context manager is that it opens up a file when you use the `with` statement, runs all your lines of indented code, and the closes the file after.  This is important if you are opening up large files.  Your computer only has so much memory, so if you try to open too many 8GB files at once without closing them you can crash your program.

If you don't want to use the contect manager another way to this same task is to run:

In [7]:
src = rasterio.open(filepath_rad, 'r')
src.read(1)
print(src)
src.close()

NameError: name 'rasterio' is not defined

If you do this, though, make sure to only be working with a few files at a time or to close your files.

### 📝 Checking In

What is happening in the following piece of code?  How many bands will be affected?

```
with rasterio.open('./data/f180628t01p00r02_corr_v1k1_img', 'r') as src:
    src.read()
```

## 🔍 Inspecting the Metadata

When you first open a dataset it is often helpful to get your bearings by inspecting the metadata.  In rasterio we do this by inspecting the DatasetReader object (`src`) that was returned when you opened the object.

There are [lots of attributes](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#rasterio._base.DatasetBase) that you check a dataset reader for.  If you want to get an general overview I like to use `src.meta`.  To extract specific values here are some more attributes that I find useful:
* [`src.bounds`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#id1) - Returns the lower left and upper right bounds of the dataset in the units of its coordinate reference system. (lower left x, lower left y, upper right x, upper right y)
* [`src.count`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#id5) - The number of raster bands in the dataset
* [`src.crs`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#id6) - The dataset’s coordinate reference system
* [`src.descriptions`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#id7) - Descriptions for each dataset band
* [`src.dtypes`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#rasterio._base.DatasetBase.dtypes) - The data types of each band in index order
* `src.height` - The number of pixels in each column
* [`src.indexes`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#id11) - The 1-based indexes of each band in the dataset
* [`src.meta`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#id14) - The basic metadata of this dataset.
* `src.name` - Relative filepath of the dataset
* [`src.nodata`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#id17) - The dataset’s single nodata value
* [`src.res`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#id22) - Returns the (width, height) of pixels in the units of its coordinate reference system.
* [`src.transform`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#id24) - The dataset’s georeferencing transformation matrix
* [`src.units`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#id25) - one units string for each dataset band
* `src.width` - the number of pixels in each row

All of the parts of the raster that were addressed in the first section of this notebook  _Anatomy of a raster_ are accessible using these attributes.

### Example

In [ ]:
# Try substituting any of the above attributes in for this dataset
# with rasterio.open(filepath_rad, 'r') as src:
#     pprint(src.meta)

In [ ]:
# with rasterio.open(filepath_rad, 'r') as src:
#     print(src.read().shape)

With the above print statement we learned from the `src.meta` information that the raster has 224 bands (`count`) it has the coordinate reference system EPSG code 32610 (`crs`), it is an ENVI filetype (`driver`), the datatype is float32 (`dtype`), it has 250 rows or is 250 pixels tall (`height`), there isn't a nodata value set (`nodata`), the affine tranform is (1.02258007728804e-15, 16.7, 769855.79, 16.7, -1.02258007728804e-15, 3751257.6) (`tranform`), and that the raster has 375 columns or is 375 pixels wide.  We also see the relative filepath of the raster from the `src.name` attribute.

### 📝 Checking In

Which of the following are metadata in an AVIRIS reflectance dataset?

- A) Number of bands
- B) Coordinate information
- C) Reflectance values
- D) Height/Lines and Width/Samples

Answer with all of the letters that are metadata.

## 🚜 Working with the Data

While the raster file is is accessed through a unique filetype-specific library, the actual data is almost always just a common matrix datatype.  Numpy is the most common library used, but xarray is also used by some data libraries.

This means that everything we working on last week for matricies applies to the data inside the raster file.

In [ ]:
# with rasterio.open(filepath_rad, 'r') as src:
#     red_band = src.read(50)
#     blue_band = src.read(10)
#     green_band = src.read(20)

### Indexing

A reminder about matrix indexing:
* 2D matrices use row, column order
* 3D matrices use height, row, column order

In [ ]:
# red_band[249, 374]

In [ ]:
# red_band[144, 100]

In [ ]:
# One long row of data
# red_band[4]

### Exploratory statistics

When I open a dataset I often like to get a sense of it by just looking at looking at statistics for one of the bands.  We can use the aggregation functions from last week for this.

In [ ]:
# print('max value: ', green_band.max())
# print('mean value: ', green_band.mean())
# print('min value: ', green_band.min())

### Matrix Operations

Adding and subtracting two matrices

In [ ]:
# red + green

In [ ]:
# blue - red

Multiplying or dividing a matrix by a constant

In [ ]:
# red*5

In [ ]:
# green/7

Some other built in operations

In [ ]:
# We have to take the absolute value of red because the nodata values are negative and that throws an error
# np.sqrt(abs(red))

In [ ]:
# np.power(red, 5)

There are lots more built in functions.  [This table](https://www.oreilly.com/library/view/python-for-data/9781449323592/ch04.html#table_unary_ufuncs) has a nice list.

### Plotting

Also notice that plotting works for a single band

In [ ]:
# pyplot.imshow(band5, cmap='pink')

### 📝 Checking In

Say that the variable `reflectance` hold all of the data for an AVIRIS raster -- count 224 with 1000 rows and 350 columns.

What is the index you need to use to get the all the data for band 100?

## 💾 Saving a file

At a certain point you may want to do it.  Let's start by getting some data to save

In [ ]:
# Get our data setup
# with rasterio.open(filepath_rad, 'r') as src:
#     red_src = src.read(50)
#     blue_src = src.read(20)
#     green_src = src.read(10)
#     meta = src.meta.copy()

It is best practice to persist or update as much of the metadata from your old raster as you can.  As you may have experienced working with data, complete and accurate metadata can make someone else's life (and maybe even your own) much easier down the road.  Make sure you look at it, though, to make sure all the information still applies to your new raster.

Count has changed, since we are only saving 3 of the 224 total bands, so we need to upadate the `count` value.  I am also going to set -50 as the nodata value.

In [ ]:
# meta['count'] = 3

Another thing you could do is add tags to your bands so that you remember what band number you used.  You do this when you write out the data.

In [ ]:
# Make the output directory if it does not exist yet
# import os
# if not os.path.exists('../output_data'):
#     os.makedirs('../output_data')

In [ ]:
# with rasterio.open(
#     '../output_data/rgb',
#     'w',
#     **meta
# ) as dst:
#     # Write data matrices
#     dst.write(red_src, 1)
#     dst.write(green_src, 2)
#     dst.write(blue_src, 3)
#     # Add band tags
#     dst.update_tags(1, src_band=50)
#     dst.update_tags(2, src_band=10)
#     dst.update_tags(3, src_band=20)

If we want to confirm that our dataset saved properly we can open it back up and look at what we saved.

In [ ]:
# with rasterio.open('../output_data/rgb', 'r') as src:
#     red_dst = src.read(1)
#     green_dst = src.read(2)
#     blue_dst = src.read(3)

In [ ]:
# print('Source stats: ')
# print('red: ', red_src.mean(), red_src.max())
# print('green: ', green_src.mean(), green_src.max())
# print('blue: ', blue_src.mean(), blue_src.max())
# print('Destination stats: ')
# print('red: ', red_dst.mean(), red_dst.max())
# print('green: ', green_dst.mean(), green_dst.max())
# print('blue: ', blue_dst.mean(), blue_dst.max())

Our values line up!  And the new values came from the raster we made ourselves.  😄